In [2]:
from bs4 import BeautifulSoup
from PyPDF2 import PdfReader
import requests
import io
import tabula
import pandas as pd
import numpy as np
from pandas import DataFrame

In [3]:
BASE_URL = "https://dijlovasok.hu/index.php/programgyujt"

In [4]:
def get_contents_of_page (url, headers = None) :
    req = requests.get(url, headers = headers if headers != None else {
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"
    })
    
    return req.content

In [5]:
def soupify (content) :
    return BeautifulSoup(content, "html.parser")

In [6]:
def get_links_from_main_page ():
    soup = soupify(get_contents_of_page(BASE_URL))
    # print(soup.prettify())
    program_hrefs = list(
        set(
            map(
                lambda href: href["href"], 
                filter(lambda href: href["href"].startswith("/index.php/programgyujt/"), 
                    soup.find_all("a", href = True)
                )
            )
        )
    )
    return program_hrefs    

In [7]:
def get_pdf_links_from_page (url):
    soup: BeautifulSoup = soupify(get_contents_of_page(url))
    pdf_hrefs = list(
            set(
                map(
                    lambda href: href["href"], 
                    filter(lambda href: href["href"].endswith(".pdf"), 
                        soup.find_all("a", href = True)
                    )
                )
            )
        )
    return pdf_hrefs

In [8]:
def get_pdf_dataframes () :

    pdf_texts = {}

    for page_url in get_links_from_main_page() :
        
        print(f"Scraping page {page_url}...")
        
        page_content = {}
        
        for pdf_url in get_pdf_links_from_page(BASE_URL + page_url) :
            
            print(f"Downloading pdf {pdf_url}...")
                        
            user_agent= 'Mozilla/5.0 (X11; Windows; Windows x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'

            dfs = tabula.read_pdf(
                pdf_url, 
                multiple_tables=True, 
                pages="all", 
                guess=True,
                encoding="cp1252",
                user_agent=user_agent,
                # output_format="dataframe",
                pandas_options={
                    # "names": ["ID", "Position", "Description", "Notes"]
                    "header": 0
                },
                lattice=True,
                silent=True
            )   
                
            if len(dfs) == 0 :
                continue
                
            page_content[pdf_url] = dfs
            
        pdf_texts[page_url] = page_content
        
    return pdf_texts

In [9]:
database = get_pdf_dataframes()

Scraping page /index.php/programgyujt/1035-kozep-osztalyu-feladatok-hatalyos-2018-05-01...
Scraping page /index.php/programgyujt/1457-kozep-osztalyu-feladatok-mb-program...
Scraping page /index.php/programgyujt/1037-nagydij-szintu-feladatok-hatalyos-2018-05-01...
Scraping page /index.php/programgyujt/1031-elokeszito-feladatok-hatalyos-2018-05-01...
Scraping page /index.php/programgyujt/1217-vezetoszaras-feladat...
Scraping page /index.php/programgyujt/1034-konnyu-kozep-osztalyu-feladatok-hatalyos-2018-05-01...
Scraping page /index.php/programgyujt/1033-konnyu-osztalyu-feladatok-hatalyos-2018-05-01...
Scraping page /index.php/programgyujt/1008-fiatal-lovak-programjai...
Scraping page /index.php/programgyujt/1032-alapfoku-feladatok-hatalyos-2018-05-01...
Scraping page /index.php/programgyujt/1036-nehez-osztalyu-feladatok-hatalyos-2018-05-01...


In [10]:
def inspect_database(database) :
    for page_url in database:
        page = database[page_url]
        for program_url in page:
            program = page[program_url]
            for df in program:
                yield df

In [11]:
next(inspect_database(database))

,Unnamed: 0


In [12]:
list(list(database.values())[0].values())[3][1]

,Unnamed: 0,Unnamed: 1,Feladat,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Megjegyzés
0,1.0,A\rX\r\rC,"Belovaglás összeszedett vágtában\rÁllj, köszön...",10,NaN,NaN,NaN,NaN,NaN
1,2.0,M-K\rK,Átlóváltás középügetésben\rÖsszeszedett ügetés,10,NaN,NaN,NaN,NaN,NaN
2,3.0,F-X,Oldaljárás balra,10,NaN,NaN,2.0,NaN,NaN
3,4.0,X-M,Oldaljárás jobbra,10,NaN,NaN,2.0,NaN,NaN
4,5.0,H-F\rF,Átlóváltás nyújtott ügetésben\rÖsszeszedett üg...,10,NaN,NaN,NaN,NaN,NaN
5,6.0,NaN,Átmenetek H-nál és F-nél,10,NaN,NaN,NaN,NaN,NaN
6,7.0,K-E\rE,Vállat be\rFordulat jobbra,10,NaN,NaN,NaN,NaN,NaN
7,8.0,X,"Állj, 5 lépést hátra lépj,\rösszeszedett ügetés",10,NaN,NaN,NaN,NaN,NaN
8,9.0,B\rB-M,Fordulat balra\rVállat be,10,NaN,NaN,NaN,NaN,NaN
9,10.0,C\rH\rG-M között,Összeszedett lépés\rFordulat balra\rFélpiruett...,10,NaN,NaN,NaN,NaN,NaN


In [13]:
from pandas import DataFrame
from typing import List

def convert_program_dfs_to_useful_df (dfs: List[DataFrame]) :
            
    dfs = list(filter(lambda df: len(df) != 0, dfs))
    
    if len(dfs) == 0 :
        return None
    
    if len(dfs) == 1 :
        if len(dfs[0]) != 0 :
            return dfs[0]
        else :
            return None
    
    # joining
    
    result = dfs[0]
    
    column_num = len(result.columns)
    
    for df in dfs[1:] :
        
        if len(df.columns) == column_num :
            result = pd.concat([result, df], ignore_index = True)
            continue
        
        if len(df.columns) == column_num + 2 :
            
            columns = df.columns
            
            df = df.drop(columns[0], axis=1)
            df = df.drop(columns[-1], axis=1)
            
            # rename unnamed columns
            
            def column_renamer(column):
                if str(column).startswith("Unnamed") == False :
                    return column
                num = int(column.split(" ")[1]) - 1
                return "Unnamed: " + str(num)
            
            df = df.rename(column_renamer, axis=1)
            
            result = pd.concat([result, df], ignore_index = True)
            continue
        
        # raise Exception(f"WARNING: df column width mismatch: cannot append {len(df.columns)} to {column_num}!")
        
    return result

In [14]:

def convert_database_to_useful_dfs (database) :
    for page in database.values() :
        for program_url in page :
            program = page[program_url]
            try :
                yield (program_url, convert_program_dfs_to_useful_df(program))
            except Exception as e :
                print(f"URL in question: {program_url}")
                raise e

In [35]:
from typing import Tuple

def is_df_program(program: Tuple[str, DataFrame]) :
    
    url, df = program
    
    if "Feladat" not in df.columns :
        
        print(f"{url} doesn't have Feladat column")
        print(f"(columns: {df.columns}, len={len(df)})")
        
        return False
    
    if len(df) <= 5 :
        
        print(f"{url} too short")
        print(f"(columns: {df.columns}, len={len(df)})")
        
        return False
    
    df_where_feladat_is_not_nan = df.dropna(inplace=False, subset=["Feladat"])
    
    if df_where_feladat_is_not_nan[df_where_feladat_is_not_nan["Feladat"].str.contains("öszönés")].count()["Feladat"] != 2 :
        
        print(f"{url} doesn't have 2 köszönés!")
        print(f"Feladat: {df['Feladat']}")
        print(f"(columns: {df.columns}, len={len(df)})")
        
        return False
    
    try :
        if int(df["Unnamed: 0"][0]) != 1 :
            
            print(f"{url} doesn't have int 'in Unnamed: 0'")
            print(f"Unnamed: 0: {df['Unnamed: 0']}")
            print(f"(columns: {df.columns}, len={len(df)})")
            
            return False
    except ValueError :
        return False
    
    return True

In [36]:
filtered_programs = list(filter(is_df_program, convert_database_to_useful_dfs(database)))

http://www.dijlovasok.hu/dokumentum/Programgyujtemeny_2018/M/Junior_zenes_kur.pdf doesn't have 2 köszönés!
Feladat: 0                     Összeszedett lépés (minimum 20 m)
1     Röviden hátra arc összeszedett lépésben\rjobbr...
2                         Nyújtott lépés (minimum 20 m)
3                                   Összeszedett ügetés
4                       Vállat be jobbra (minimum 12 m)
5                        Vállat be balra (minimum 12 m)
6            Féloldalazás jobbra összeszedett ügetésben
7             Féloldalazás balra összeszedett ügetésben
8                                       Nyújtott ügetés
9                                    Összeszedett vágta
10            Féloldalazás jobbra összeszedett vágtában
11             Féloldalazás balra összeszedett vágtában
12                            Ugrásváltás balról jobbra
13                            Ugrásváltás jobbról balra
14                                       Nyújtott vágta
15    A belovaglás és az állj a program elej

http://www.dijlovasok.hu/dokumentum/Programgyujtemeny_2018/M/LSZ4_kulon.pdf doesn't have 2 köszönés!
Feladat: 0     Belovaglás összeszedett ügetésben\rÁllj, köszö...
1          Átlóváltás, középügetés\rÖsszeszedett ügetés
2     Átmenetek összeszedett ügetésbõl\rközépügetésb...
3                                             Vállat be
4                                           Kiskör, 10m
5                           Oldaljárás balra\rBal kézre
6          Átlóváltás, középügetés\rÖsszeszedett ügetés
7     Állj, 5 lépést hátralépj, abból\rÖsszeszedett ...
8                                             Vállat be
9                                           Kiskör, 10m
10                                    Oldaljárás jobbra
11                               Középlépés\rJobb kézre
12            Fordulat jobbra\rRöviden hátra arc jobbra
13                              Röviden hátra arc balra
14                                       (A középlépés)
15            Félátlóváltás, nyújtott lépés\rKözép

In [37]:
def rename_columns (program: Tuple[str, DataFrame]) :
    url, df = program
    df = df.rename({
        "Unnamed: 0": "Id",
        "Unnamed: 1": "Letter"
    }, axis=1, inplace=False)
    return (url, df)

In [38]:
programs = list(map(rename_columns, filtered_programs))

In [39]:
def keep_only_useful_columns (program: Tuple[str, DataFrame]) :
    url, df = program
    columns = df.columns
    columns = filter(lambda c: c not in ["Id", "Letter", "Feladat"], columns)
    for column in columns :
        df = df.drop(column, axis = 1)
    return (url, df)

In [40]:
programs = list(map(keep_only_useful_columns, programs))

In [41]:
def keep_only_useful_rows (program: Tuple[str, DataFrame]) :
    url, df = program
    return (url, df.dropna(axis=0, how="any", subset=["Id"]))

In [42]:
programs = list(map(keep_only_useful_rows, programs))

In [43]:
len(programs)

79

In [44]:
def tokenize_exercise (letters: str, exercise: str) :
    
    if str(exercise) != "nan" :
        rows = str(exercise).split("\r")
        words = [str(row).split() for row in rows]
    else :
        words = []
    
    if str(letters) != "nan" :
        letter_rows = str(letters).split('\r')
        letter_words = [str(letter_row).split() for letter_row in letter_rows]
    else :
        letter_words = []
    
    return (letter_words, words)

In [45]:
def tokenize_program(program: Tuple[str, DataFrame]) :
    url, df = program
    tokens = []
    for index, row in df.iterrows() :
        tokens.append(tokenize_exercise(row["Letter"], row["Feladat"]))
    return (url, tokens)

In [46]:
programs = [tokenize_program(program) for program in programs]

In [47]:
programs[3]

('https://www.dijlovasok.hu/dokumentum/Programgyujtemeny_2018/M/M9.pdf',
 [([['A'], ['X'], [], ['C']],
   [['Belovaglás', 'összeszedett', 'vágtában'],
    ['Állj,', 'köszönés,', 'elindulás'],
    ['összeszedett', 'ügetésben'],
    ['Jobb', 'kézre']]),
  ([['M-K'], ['K']],
   [['Átlóváltás', 'középügetésben'], ['Összeszedett', 'ügetés']]),
  ([['F-X']], [['Oldaljárás', 'balra']]),
  ([['X-M']], [['Oldaljárás', 'jobbra']]),
  ([['H-F'], ['F']],
   [['Átlóváltás', 'nyújtott', 'ügetésben'], ['Összeszedett', 'ügetés']]),
  ([], [['Átmenetek', 'H-nál', 'és', 'F-nél']]),
  ([['K-E'], ['E']], [['Vállat', 'be'], ['Fordulat', 'jobbra']]),
  ([['X']],
   [['Állj,', '5', 'lépést', 'hátra', 'lépj,'], ['összeszedett', 'ügetés']]),
  ([['B'], ['B-M']], [['Fordulat', 'balra'], ['Vállat', 'be']]),
  ([['C'], ['H'], ['G-M', 'között']],
   [['Összeszedett', 'lépés'],
    ['Fordulat', 'balra'],
    ['Félpiruett', 'balra']]),
  ([['G-H', 'között'], ['M']], [['Félpiruett', 'jobbra'], ['Jobb', 'kézre']]),
  

In [53]:
def get_program_id_from_url (url) :
    return url.split("/")[-1].split(".pdf")[0]

In [54]:
programs = [(get_program_id_from_url(url), df) for url, df in programs]

In [58]:
programs[50]

('L1',
 [([['A'], ['X'], ['C']],
   [['Belovaglás', 'munkaügetésben'],
    ['Állj,', 'köszönés,', 'elindulás', 'összeszedett'],
    ['ügetésben'],
    ['Bal', 'kézre']]),
  ([['E']], [['Kiskör,', '10', 'm']]),
  ([['F-H'], ['H']],
   [['Átlóváltás', 'középügetésben'], ['Összeszedett', 'ügetés']]),
  ([['B']], [['Kiskör,', '10', 'm']]),
  ([['A']],
   [['Állj,', 'egy', 'lóhossznyit', 'hátra', 'lépj,', 'majd', 'középlépés']]),
  ([['E']],
   [['Hátulja', 'körüli', 'fordulat', 'jobbra,', 'majd', 'középlépés']]),
  ([['K']],
   [['Hátulja', 'körüli', 'fordulat', 'balra,', 'majd', 'középlépés']]),
  ([['E'], ['(AEKEX)']], [['Fordulat', 'jobbra'], ['(Középlépés)']]),
  ([['X'], ['B']],
   [['Összeszedett', 'vágta', 'jobb', 'kézre'], ['Fordulat', 'jobbra']]),
  ([['F-D-B'], ['B-M']], [['Átváltás'], ['Ellenvágta']]),
  ([['M']], [['Egyszerû', 'ugrásváltás']]),
  ([['E']], [['Kiskör,', '10', 'm']]),
  ([['K-D-E'], ['E-H']], [['Átváltás'], ['Ellenvágta']]),
  ([['H']], [['Egyszerû', 'ugrásváltás